In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
from ydata_profiling import ProfileReport

c:\Users\jimmy\miniconda3\envs\minimal_ds2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jimmy\miniconda3\envs\minimal_ds2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from config import sheet_id, sheet_name

In [4]:
import os
os.getcwd()

'c:\\Users\\jimmy\\Documents\\Data Science Folder\\basecamp_salary_dashboard'

In [5]:
# import sheet_id and sheet_name from a local config file
sheet_id = sheet_id
sheet_name = sheet_name
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
# replace whitespace in the URL
url = url.replace(" ", "%20")

In [6]:
# specify columns to read in (all but the "Company Name" column)
# parse_dates argument tells us which column is a datetime field
df_raw = pd.read_csv(url, usecols = lambda x: x != "Company Name", parse_dates = ['Submitted At'])

In [7]:
df_raw.head()

,I am a:,Industry,Job Title,Location,Flexibility,# Years,Pay,I charge per:,Bonuses/ Commissions,Equity,...,Field of work,City,Government role?,Parental leave,Pronouns,Race/Ethnicity,I identify as part of the following communities,Submitted At,Token,Unnamed: 27
0,Full-Time Employee,Active Lifestyle,Vice President,California,Fully Remote,25+,0,/Year,2000.0,False,...,Marketing & Branding,NaN,Nope,3 months,She/Her,White/Caucasian,Mom,2022-10-31 15:14:23,n9h9wsaeudjerl5v9cn9h9wlg4wo67z7,NaN
1,Full-Time Employee,Bike,Operations Manager,Colorado,In-Office,1,20,/Year,3000.0,False,...,Marketing & Branding,Denver,Local,None,She/Her,White/Caucasian,Woman,2023-01-19 21:19:56,i50qp8bzcbk5lns69li50qpl0z4xix5z,NaN
2,Full-Time Employee,Camp Jobs,Instructor,Colorado,In-Office,15+,1500,/Year,0.0,False,...,Administrative/Virtual Assistance,Denver,State,None,They/Them,White/Caucasian,Woman,2022-11-01 16:34:22,pgfw9l4162v9hfuf6ihhudpgfw9ljz9p,NaN
3,Contract/Seasonal Worker,Non-Profit,Field instructor,Oregon,In-Office,3,15000,/Year,0.0,False,...,Guiding or Instructing,NaN,Nope,No idea,He/Him,"Black, White/Caucasian",NaN,2022-10-31 20:36:11,fei6mrumub2az0h91nfei6mruh93op2n,NaN
4,Intern,Education,Teaching Fellow,Idaho,In-Office,1,15000,/Year,NaN,False,...,Education,Sun Valley,Nope,No idea,He/Him,White/Caucasian,NaN,2023-03-09 22:29:01,b3opzb9dq7c3tzdb3opzbu62jbsry1ku,NaN


In [8]:
df = df_raw.copy(deep = True)

In [9]:
df.columns

Index(['I am a:', 'Industry', 'Job Title', 'Location', 'Flexibility',
       '# Years', 'Pay', 'I charge per:', 'Bonuses/ Commissions', 'Equity',
       'PTO', 'Training/Education', 'Add'l Qs', 'Benefits', 'OTJ Experience',
       'Starting Pay', 'Specialty expertise', 'Field of work', 'City',
       'Government role?', 'Parental leave ', 'Pronouns', 'Race/Ethnicity',
       'I identify as part of the following communities', 'Submitted At',
       'Token', 'Unnamed: 27'],
      dtype='object')

In [10]:
# general syntax of loc
# df.loc['row_label', 'column_label']

# If there are multiple labels, they should be specified inside lists:
# df.loc[['row_1', 'row_2'], ['column_1', 'column_2']]


In [11]:
# specify which columns to keep
columns_wanted = ['I am a:', 'Industry', 'Location', 'Flexibility', '# Years', 'Pay', 'I charge per:', 'Field of work', 'Submitted At', 'Pronouns']

In [12]:
# create a subset of the DataFrame with specified columns
df_subset = df.loc[:, columns_wanted]

In [13]:
df_subset.head()

,I am a:,Industry,Location,Flexibility,# Years,Pay,I charge per:,Field of work,Submitted At,Pronouns
0,Full-Time Employee,Active Lifestyle,California,Fully Remote,25+,0,/Year,Marketing & Branding,2022-10-31 15:14:23,She/Her
1,Full-Time Employee,Bike,Colorado,In-Office,1,20,/Year,Marketing & Branding,2023-01-19 21:19:56,She/Her
2,Full-Time Employee,Camp Jobs,Colorado,In-Office,15+,1500,/Year,Administrative/Virtual Assistance,2022-11-01 16:34:22,They/Them
3,Contract/Seasonal Worker,Non-Profit,Oregon,In-Office,3,15000,/Year,Guiding or Instructing,2022-10-31 20:36:11,He/Him
4,Intern,Education,Idaho,In-Office,1,15000,/Year,Education,2023-03-09 22:29:01,He/Him


In [14]:
df_subset["I am a:"].value_counts()

Full-Time Employee          670
Contract/Seasonal Worker     11
Part-Time Employee            4
Intern                        1
Name: I am a:, dtype: int64

In [15]:
df_subset["I charge per:"].value_counts()

/Year    686
Name: I charge per:, dtype: int64

In [16]:
# check datatypes
df_subset.dtypes

I am a:                  object
Industry                 object
Location                 object
Flexibility              object
# Years                  object
Pay                       int64
I charge per:            object
Field of work            object
Submitted At     datetime64[ns]
Pronouns                 object
dtype: object

In [17]:
# subset only responses with 'I charge per: ' '/Year'
df_subset = df_subset.loc[df_subset['I charge per:'] == '/Year']

In [18]:
# Extract the Year from the Submitted At column
df_subset['Year'] = df_subset['Submitted At'].dt.strftime('%Y')

In [19]:
# Rename columns
df_subset.rename(columns = {"# Years": "Years of Experience"}, inplace = True)

In [20]:
df_subset['Years of Experience'].value_counts()

10+                 100
2                    90
1                    72
Less than a year     71
3                    65
4                    63
5                    57
15+                  39
7                    32
6                    29
8                    24
9                    19
20+                  18
25+                   6
30+                   1
Name: Years of Experience, dtype: int64

In [21]:
# Create categories for "Years of Experience"
df_subset['Years of Exp.'] = np.select(
    [
        df_subset["Years of Experience"] == 'Less than a year',
        df_subset["Years of Experience"] == '1',
        df_subset["Years of Experience"] == '2',
        df_subset["Years of Experience"] == '3',
        df_subset["Years of Experience"] == '4',
        df_subset["Years of Experience"] == '5',
        df_subset["Years of Experience"] == '6',
        df_subset["Years of Experience"] == '7',
        df_subset["Years of Experience"] == '8',
        df_subset["Years of Experience"] == '9',
        df_subset["Years of Experience"] == '10+',
        df_subset["Years of Experience"] == '15+',
        df_subset["Years of Experience"] == '20+',
        df_subset["Years of Experience"] == '25+',
        df_subset["Years of Experience"] == '23+' 
    ],
    [
        '0-3 years',
        '0-3 years',
        '0-3 years',
        '0-3 years',
        '4-6 years',
        '4-6 years',
        '4-6 years',
        '7-9 years',
        '7-9 years',
        '7-9 years',
        '10+ years',
        '10+ years',
        '10+ years',
        '10+ years',
        '10+ years'
    ],
    default = 'No Data'
)

In [22]:
df_subset["Pronouns"].value_counts()

She/Her      434
He/Him       217
They/Them      7
she/they       2
She/they       1
She/They       1
Name: Pronouns, dtype: int64

In [23]:
# Create categories for "Pronouns"
df_subset['Pronouns'] = np.select(
    [
        df_subset["Pronouns"] == 'She/Her',
        df_subset["Pronouns"] == 'He/Him',
        df_subset["Pronouns"] == 'They/Them',
        df_subset["Pronouns"] == 'she/they',
        df_subset["Pronouns"] == 'She/They',
        df_subset["Pronouns"] == 'She/they'
    ],
    [
        'She/Her',
        'He/Him',
        'They/Them',
        'She/They',
        'She/They',
        'She/They'
    ],
    default = 'No Response'
)

In [24]:
df_subset.head(10)

,I am a:,Industry,Location,Flexibility,Years of Experience,Pay,I charge per:,Field of work,Submitted At,Pronouns,Year,Years of Exp.
0,Full-Time Employee,Active Lifestyle,California,Fully Remote,25+,0,/Year,Marketing & Branding,2022-10-31 15:14:23,She/Her,2022,10+ years
1,Full-Time Employee,Bike,Colorado,In-Office,1,20,/Year,Marketing & Branding,2023-01-19 21:19:56,She/Her,2023,0-3 years
2,Full-Time Employee,Camp Jobs,Colorado,In-Office,15+,1500,/Year,Administrative/Virtual Assistance,2022-11-01 16:34:22,They/Them,2022,10+ years
3,Contract/Seasonal Worker,Non-Profit,Oregon,In-Office,3,15000,/Year,Guiding or Instructing,2022-10-31 20:36:11,He/Him,2022,0-3 years
4,Intern,Education,Idaho,In-Office,1,15000,/Year,Education,2023-03-09 22:29:01,He/Him,2023,0-3 years
5,Contract/Seasonal Worker,Non-Profit,Washington State,Hybrid/Work From Home,4,20000,/Year,Guiding or Instructing,2022-11-08 03:55:00,She/Her,2022,4-6 years
6,Full-Time Employee,Bike,Spain,In-Office,2,21000,/Year,Marketing & Branding,2022-10-31 16:25:02,He/Him,2022,0-3 years
7,Full-Time Employee,Outdoor (Multi-Industry Apparel/ Gear/Services),United Kingdom,Hybrid/Work From Home,Less than a year,24000,/Year,Marketing & Branding,2022-10-31 21:17:35,She/Her,2022,0-3 years
8,Full-Time Employee,Outdoor Adjacent,Washington State,In-Office,3,25000,/Year,Guiding or Instructing,2022-11-01 06:09:00,She/Her,2022,0-3 years
9,Full-Time Employee,Camp Jobs,Vermont,Hybrid/Work From Home,2,25000,/Year,Development and Operations,2022-11-09 23:00:08,He/Him,2022,0-3 years


In [25]:
# remove outliers with pay less than $100 per year
index_sal_100 = df_subset[(df_subset['Pay'] <= 100)].index
df_subset.drop(index_sal_100 , inplace=True)

In [26]:
df_subset.to_csv("salary_data.csv")

In [27]:
df_subset.Pay.describe()

count       684.000000
mean      75797.475146
std       35808.735384
min        1500.000000
25%       53000.000000
50%       68000.000000
75%       88125.000000
max      403950.000000
Name: Pay, dtype: float64

In [28]:
df_subset.Year.value_counts()

2022    575
2023    109
Name: Year, dtype: int64

In [31]:
profile = ProfileReport(df_subset, title = "Profiling Report")

In [32]:
profile.to_file("data_quality_report.html")

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Only supported for TrueType fonts